In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Transformer, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical



ImportError: cannot import name 'Transformer' from 'tensorflow.keras.layers' (C:\Users\karan\AppData\Roaming\Python\Python310\site-packages\keras\api\_v2\keras\layers\__init__.py)

In [6]:
#ATS System
df = pd.read_csv('UpdatedResumeDataSet.csv')
df.head()
max_len = 300
resumes = df['resume'].tolist()
categories = df['category'].tolist()
Tokenizer = Tokenizer()
Tokenizer.fit_on_texts(resumes)
sequences = Tokenizer.texts_to_sequences(df['Resume'].values)

X = pad_sequences(sequences, maxlen=max_len)
y = pd.get_dummies(df['Category']).values

categories_encoded = to_categorical(categories)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


KeyError: 'resume'

In [ ]:
embeddings_dim = 100
embeddings_index = {}
with open('glove.6B.100d.txt', encoding = 'utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embedding_matrix = np.zeros((len(Tokenizer.word_index) + 1, embeddings_dim))
for word, i in Tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

model = Sequential([
    Embedding(len(Tokenizer.word_index) + 1, embeddings_dim, input_length=max_len, weights=[embedding_matrix], trainable=False),
    Transformer(num_layers=2, units=128, head_size=8),
    Dense(len(categories), activation='softmax')

])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64, callbacks=[EarlyStopping(patience=2)])
loss,auc = model.evaluate(X_test, y_test)
print(f"Loss: {loss}")

new_resume = "I am a software engineer with 5 years of experience in Java, Python, and C++. I have a Bachelor's degree in Computer Science from MIT."
new_resume = Tokenizer.texts_to_sequences([new_resume])
new_resume = pad_sequences(new_resume, maxlen=max_len)
prediction = model.predict(new_resume)
print(f"Predicted category: {categories[np.argmax(prediction)]}")
